# Generate interval best execution benchmarks using dedicated service - C#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `interval` approach and designed to illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the point-in-time approach, suitable to highlight individual tradres among market activity. Please refer to <i>bestex-pointintime</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *interval* benchmarks.

### Inputs/outputs
Best execution interval sample requires instrument's identifier, date time intervals as per inputs. Results are as follows:
* a snapshot of occured trades
* a set of computed benchmarks based on the occured trades

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run BestEx interval sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: CsvHelper"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages CsvHelper, 29.0.0 Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Plot charts
using XPlot.Plotly; 
// Import/export csv files 
using CsvHelper;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [4]:
// Instrument
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [5]:
// Create the date interval (we are using Google date time format)
var day = new DateTime(2021, 03, 15);
var dateIntervals = new DateInterval()
{
    StartDate = new Date { Year = 2021, Month = 11, Day = 15 },
    EndDate = new Date { Year = 2021, Month =11, Day = 15 }
};

// Create the time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{
    StartTime = new TimeOfDay { Hours = 16, Minutes = 20, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 16, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [6]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create request and instantiate the service:

In [7]:
// Generate the tick trades request
var request = new TickTradesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [8]:
// Instantiate the tick trades service
var service = new TickTradesService.TickTradesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve trade prices and volumes

In [9]:
// Get the trades 
// store the list of trades
var trades = new List<Tuple<Timestamp, Trade>>();

// Call the service
var call = service.TickTrades(request, headers);

// Process the responses
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if(current.Mapping != null)
    {
        // First response contains the mapping fields
        // Skip the mapping data
        continue;
    }

    // Get the time stamp for the current trade
    var ts = current.Data.TimeStamp;

    // Trade
    if (current.Data.Trade != null) 
    {      
        trades.Add(new Tuple<Timestamp, Trade>(ts, current.Data.Trade));
    }
}

The following code snippet displays request results:

In [10]:
// Display the trades
//--> Note : the time stamp is displayed by default (using the unix format)
//--> We can use other displayers for different output formats 
//--> For more details about the time stamp: https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp 
//display(trades);

#### 4.2 Retrieve interval benchmarks

In [11]:
// Get bestex time interval
var intervalStart = new DateTime(dateIntervals.StartDate.Year, dateIntervals.StartDate.Month, dateIntervals.StartDate.Day, timeInterval.StartTime.Hours, timeInterval.StartTime.Minutes, timeInterval.StartTime.Seconds);
var intervalEnd = new DateTime(dateIntervals.EndDate.Year, dateIntervals.EndDate.Month, dateIntervals.EndDate.Day, timeInterval.EndTime.Hours, timeInterval.EndTime.Minutes, timeInterval.EndTime.Seconds);

// Get first and last trades' timestamps
var firstTimestamp = trades.Select(elt => elt.Item1.ToDateTime()).First();
var lastTimestamp = trades.Select(elt => elt.Item1.ToDateTime()).Last();

// Get a set of benchmarks
var totalTrades = trades.Count;
var totalSizes = trades.Sum(trade => trade.Item2.Size);
var avgPrice = trades.Select(elt => elt.Item2.Price).Average();
var avgSize = trades.Select(elt => elt.Item2.Size).Average();
var maxPrice = trades.Max(trade => trade.Item2.Price);
var minPrice = trades.Min(trade => trade.Item2.Price);
var maxSize = trades.Max(trade => trade.Item2.Size);
var minSize = trades.Min(trade => trade.Item2.Size);
var vwap = trades.Sum(trade => trade.Item2.Price * trade.Item2.Size ) / totalSizes;

In [12]:
// Display results
Console.WriteLine($"Interval start time \t: {intervalStart:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Interval end time\t: {intervalEnd:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"First trade occured at\t: {firstTimestamp:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Last trade occured at\t: {lastTimestamp:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Total trades retrieved\t: {trades.Count:N0}");
Console.WriteLine($"Cumulated trade sizes\t: {totalSizes:N0}");
Console.WriteLine($"Average trade price\t: {avgPrice:F3}");
Console.WriteLine($"Average trade size\t: {avgSize:N0}");
Console.WriteLine($"Max trade price\t\t: {maxPrice:F3}");
Console.WriteLine($"Min trade price\t\t: {minPrice:F3}");
Console.WriteLine($"Max trade size\t\t: {maxSize:N0}");
Console.WriteLine($"Min trade size\t\t: {minSize:N0}");
Console.WriteLine($"Vwap\t\t\t: {vwap:F3}");

Interval start time 	: 2021-11-15 16:20:00


Interval end time	: 2021-11-15 16:30:00


First trade occured at	: 2021-11-15 16:20:00


Last trade occured at	: 2021-11-15 16:29:58


Total trades retrieved	: 394


...

Max trade price		: 150.130


Min trade price		: 149.960


Max trade size		: 600


Min trade size		: 1


Vwap			: 150.047


#### 4.3 Plot the interval trades
Display trade prices and sizes over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [13]:
var pricesAndVolumes = new List<Scattergl>
{ 
    new Scattergl
    {
        name = "Tradeprice",
        mode = "lines",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Price)
    },
    new Scattergl
    {
        name = "Tradesize",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Size),
        yaxis = "y2"
    }
};


var layout = new Layout.Layout{ 
    title = $"Interval trades for {ticker}-{exchange} | Trades count: {totalTrades} | Toal traded volume: {totalSizes} ",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Size", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(pricesAndVolumes, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>